In [1]:
import numpy as np
import pandas as pd
import pennylane as q
import scipy.stats as s
import copy
import datetime

from QuantumOperations import ClassicalOperations, QuantumGates, QuantumAlgorithms

In [2]:
c = ClassicalOperations()
qg = QuantumGates()
qa = QuantumAlgorithms()

## controlled_Two_level_U

In [3]:
# rows of U with non-trivial entries
non_trivial_indices = np.array([4,2])

In [4]:
U = s.unitary_group.rvs(2)
# U = np.array([[0,1],
#               [1,0]])
# get V from V^4 = U
V = np.eye(2**3,dtype='complex128')
for i in range(2):
    for j in range(2):
        V[non_trivial_indices[i]][non_trivial_indices[j]] = U[i][j]

print('Matrix U')
print(U)
print('\033[1m'+'probabilities for U|0>:'+str(np.absolute(U.dot(np.array([1,0])))**2)+'\033[0m')
print('\033[1m'+'probabilities for U|1>:'+str(np.absolute(U.dot(np.array([0,1])))**2)+'\033[0m')

print('Matrix V')
print(V)

Matrix U
[[ 0.63684606+0.62372668j -0.11349255-0.43876139j]
 [-0.28983612+0.34840657j -0.84208224+0.29241303j]]
probabilities for U|0>:[0.79460789 0.20539211]
probabilities for U|1>:[0.20539211 0.79460789]
Matrix V
[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          1.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.84208224+0.29241303j
   0.        +0.j         -0.28983612+0.34840657j  0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j    

In [5]:
# wires
wires=['q0','q1','q2','q3']
# device
dev = q.device('default.qubit', wires=wires, shots=1e6, analytic=False)

# circuit
def func(V,non_trivial_indices,input_):
    
    # preparation
    if (int(input_[0].val) == 1):
        q.PauliX(wires=wires[0])
    if (int(input_[1].val) == 1):
        q.PauliX(wires=wires[1])
    if (int(input_[2].val) == 1):
        q.PauliX(wires=wires[2])
    if (int(input_[3].val) == 1):
        q.PauliX(wires=wires[3])
    
    qg.controlled_Two_level_U(V,non_trivial_indices,wires[0],wires[1:])
    
    return q.probs(wires)

# QNode
circuit = q.QNode(func,dev)

In [6]:
# Check input 
states = c.states_vector(wires)
measurements = circuit(V,non_trivial_indices,np.array([1,0,0,1]))
np.vstack([states,measurements]).T

array([['|0000>', '0.0'],
       ['|0001>', '0.0'],
       ['|0010>', '0.0'],
       ['|0011>', '0.0'],
       ['|0100>', '0.0'],
       ['|0101>', '0.0'],
       ['|0110>', '0.0'],
       ['|0111>', '0.0'],
       ['|1000>', '0.0'],
       ['|1001>', '1.0'],
       ['|1010>', '0.0'],
       ['|1011>', '0.0'],
       ['|1100>', '0.0'],
       ['|1101>', '0.0'],
       ['|1110>', '0.0'],
       ['|1111>', '0.0']], dtype='<U32')

In [7]:
start = datetime.datetime.now()

# Full computational basis check
df = pd.DataFrame(c.states_vector(wires),columns=['states'])
for i in range(2**len(wires)):
    input_str = '0'*(len(wires)-len(bin(i)[2:])) + bin(i)[2:]
    input_np_array = np.array([int(input_str[j]) for j in range(len(input_str))])
    
    measurements = circuit(V,non_trivial_indices,input_np_array)
    
    df[input_str] = measurements

end = datetime.datetime.now()
print(end-start)

0:00:07.812806


In [8]:
df

,states,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,|0000>,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,|0001>,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,|0010>,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
3,|0011>,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4,|0100>,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
5,|0101>,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
6,|0110>,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
7,|0111>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
8,|1000>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
9,|1001>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,0.0,0.0


# C_U_n

In [9]:
U = s.unitary_group.rvs(8)

print('Matrix U')
print(U)
print('\033[1m'+'probabilities for U|000>:'+str(np.absolute(U.dot(np.array([1,0,0,0,0,0,0,0])))**2)+'\033[0m')
print('\033[1m'+'probabilities for U|111>:'+str(np.absolute(U.dot(np.array([0,0,0,0,0,0,0,1])))**2)+'\033[0m')

Matrix U
[[ 0.18214014-0.08145098j  0.10871283+0.48400087j  0.39785714-0.09065597j
  -0.13571506-0.34866807j -0.06793573+0.00548285j -0.52872786-0.13372107j
  -0.25679914-0.08534556j  0.17784635+0.02607282j]
 [ 0.56763078-0.07488335j -0.09265258+0.23415827j  0.02115846-0.23137584j
   0.25271419-0.13271346j -0.2045852 +0.12196612j  0.43270683+0.14883434j
   0.08809579+0.39717973j  0.19814072+0.04922763j]
 [ 0.23471815+0.02779363j  0.40571297-0.0075634j   0.00953835+0.1695155j
   0.31346964-0.09774707j  0.3455883 +0.08965583j  0.04136018+0.38247454j
   0.1432553 -0.52336737j  0.1174049 +0.24319584j]
 [-0.01330239-0.04615284j -0.18689639+0.32550663j -0.36474154-0.00360927j
   0.0605563 +0.376494j   -0.07177487+0.20054576j  0.05030797+0.05655038j
  -0.34165544-0.3562887j   0.36969093-0.38327978j]
 [ 0.26142977-0.16352499j  0.25761194+0.09691832j -0.08438964+0.12049975j
  -0.25414797+0.4924869j   0.09798632-0.04754172j  0.11217518-0.32273817j
  -0.2814199 +0.08716571j -0.05472359+0.53102375

In [10]:
# wires
wires=['q0','q1','q2','q3']
# device
dev = q.device('default.qubit', wires=wires, shots=1e6, analytic=False)

# circuit
def func(V,input_):
    
    # preparation
    if (int(input_[0].val) == 1):
        q.PauliX(wires=wires[0])
    if (int(input_[1].val) == 1):
        q.PauliX(wires=wires[1])
    if (int(input_[2].val) == 1):
        q.PauliX(wires=wires[2])
    if (int(input_[3].val) == 1):
        q.PauliX(wires=wires[3])
    
    qg.C_U_n(V,wires[0],wires[1:])
    
    return q.probs(wires)

# QNode
circuit = q.QNode(func,dev)

In [11]:
# Check input
states = c.states_vector(wires)
measurements = circuit(U,np.array([1,0,0,0]))
np.vstack([states,measurements]).T

array([['|0000>', '0.0'],
       ['|0001>', '0.0'],
       ['|0010>', '0.0'],
       ['|0011>', '0.0'],
       ['|0100>', '0.0'],
       ['|0101>', '0.0'],
       ['|0110>', '0.0'],
       ['|0111>', '0.0'],
       ['|1000>', '0.039422'],
       ['|1001>', '0.327831'],
       ['|1010>', '0.056063'],
       ['|1011>', '0.002315'],
       ['|1100>', '0.094898'],
       ['|1101>', '0.421304'],
       ['|1110>', '0.016357'],
       ['|1111>', '0.04181']], dtype='<U32')

In [12]:
start = datetime.datetime.now()

# Full computational basis check
df = pd.DataFrame(c.states_vector(wires),columns=['states'])
for i in range(2**len(wires)):
    input_str = '0'*(len(wires)-len(bin(i)[2:])) + bin(i)[2:]
    input_np_array = np.array([int(input_str[j]) for j in range(len(input_str))])
    
    measurements = circuit(U,input_np_array)
    
    df[input_str] = measurements
df

end = datetime.datetime.now()
print(end-start)

0:02:33.097406


In [13]:
df

,states,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,|0000>,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,|0001>,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,|0010>,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,|0011>,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,|0100>,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,|0101>,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,|0110>,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,|0111>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,|1000>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039756,0.245520,0.166260,0.140054,0.004564,0.296874,0.073174,0.032265
9,|1001>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327868,0.063793,0.054320,0.081457,0.056616,0.209903,0.166366,0.041794


## CR_k

In [14]:
k=3

In [15]:
# wires are standard - q.enable_tape() doesn't work with custom names of wires for some reason
wires=[0,1]
# device
dev = q.device('default.qubit', wires=wires)
# to enable experimental mode in which it is possible to return quantum states of qubits
q.enable_tape()

# circuit
def func(k,input_):
    
    # preparation
    if (input_[0] == 1):
        q.PauliX(wires=wires[0])
    if (input_[1] == 1):
        q.PauliX(wires=wires[1])
    
    qg.CR_k(k,wires[0],wires[1])
    
    return q.state()

# QNode
circuit = q.QNode(func,dev)

In [16]:
# Check input
states = c.states_vector(wires)
measurements = circuit(k,np.array([1,0]))
np.vstack([states,measurements]).T

array([['|00>', '0j'],
       ['|01>', '0j'],
       ['|10>', '(1+5.551115123125783e-17j)'],
       ['|11>', '0j']], dtype='<U64')

In [17]:
start = datetime.datetime.now()

# Full computational basis check
df = pd.DataFrame(c.states_vector(wires),columns=['states'])
for i in range(2**len(wires)):
    input_str = '0'*(len(wires)-len(bin(i)[2:])) + bin(i)[2:]
    input_np_array = np.array([int(input_str[j]) for j in range(len(input_str))])
    
    measurements = circuit(k,input_np_array)
    
    df[input_str] = measurements

end = datetime.datetime.now()
print(end-start)

0:00:00.009974


In [18]:
df

,states,00,01,10,11
0,|00>,(1+0j),0j,0j,0j
1,|01>,0j,(1+0j),0j,0j
2,|10>,0j,0j,(1+5.551115123125783e-17j),0j
3,|11>,0j,0j,0j,(0.7071067811865475+0.7071067811865475j)


In [19]:
q.disable_tape()

## QFT

In [20]:
# wires are standard - q.enable_tape() doesn't work with custom names of wires for some reason
wires=[0,1]
# device
dev = q.device('default.qubit', wires=wires)
# to enable experimental mode in which it is possible to return quantum states of qubits
q.enable_tape()

# circuit
def func(input_):
    
    # preparation
    if (input_[0] == 1):
        q.PauliX(wires=wires[0])
    if (input_[1] == 1):
        q.PauliX(wires=wires[1])
    
    qa.QFT(wires=wires)
    
    return q.state()

# QNode
circuit = q.QNode(func,dev)

In [21]:
# Check input
states = c.states_vector(wires)
measurements = circuit(np.array([0,0]))
np.vstack([states,measurements]).T

array([['|00>', '(0.49999999999999983+1.9626155733547187e-17j)'],
       ['|01>', '(0.49999999999999983-1.9626155733547187e-17j)'],
       ['|10>', '(0.49999999999999983+1.9626155733547187e-17j)'],
       ['|11>', '(0.49999999999999983-1.9626155733547187e-17j)']],
      dtype='<U64')

In [22]:
start = datetime.datetime.now()

# Full computational basis check
df = pd.DataFrame(c.states_vector(wires),columns=['states'])
for i in range(2**len(wires)):
    input_str = '0'*(len(wires)-len(bin(i)[2:])) + bin(i)[2:]
    input_np_array = np.array([int(input_str[j]) for j in range(len(input_str))])
    
    measurements = circuit(input_np_array)
    
    df[input_str] = measurements

end = datetime.datetime.now()
print(end-start)

0:00:00.015345


In [23]:
# expected output:
# [[0.5+0.0j,  0.5+0.0j,  0.5+0.0j,  0.5+0.0j],
#  [0.5+0.0j,  0.0+0.5j, -0.5+0.0j,  0.0-0.5j],
#  [0.5+0.0j, -0.5+0.0j,  0.5+0.0j, -0.5+0.0j],
#  [0.5+0.0j,  0.0-0.5j, -0.5+0.0j,  0.0+0.5j]]
df

,states,00,01,10,11
0,|00>,(0.49999999999999983+1.9626155733547187e-17j),(0.4999999999999999+0j),(0.49999999999999983+1.9626155733547187e-17j),(0.4999999999999999+0j)
1,|01>,(0.49999999999999983-1.9626155733547187e-17j),0.4999999999999999j,(-0.49999999999999983+1.9626155733547187e-17j),-0.4999999999999999j
2,|10>,(0.49999999999999983+1.9626155733547187e-17j),(-0.4999999999999999+0j),(0.49999999999999983+1.9626155733547187e-17j),(-0.4999999999999999+0j)
3,|11>,(0.49999999999999983-1.9626155733547187e-17j),-0.4999999999999999j,(-0.49999999999999983+1.9626155733547187e-17j),0.4999999999999999j


In [24]:
q.disable_tape()

# Phase_Estimation

#### Setting 1
First register contains 1 qubit\
Second register contains 1 qubit
$$U = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$$
The first register is prepared in a state$|0\rangle$\
The second register is prepared in one of two eigenstates of U:
$$|u_1\rangle = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ 1 \end{bmatrix}\text{ , }|u_2\rangle = \frac{1}{\sqrt{2}} \begin{bmatrix} 1 \\ -1 \end{bmatrix},$$

In [25]:
t = 1

In [26]:
# wires
wires=['q0','q1']
# device
dev = q.device('default.qubit', wires=wires, shots=1e6, analytic=False)

# circuit
def func(U,eigenstate):
    
    # preparation
    if eigenstate.val == 1:
        q.Hadamard(wires=wires[1])
    if eigenstate.val == 2:
        q.PauliX(wires=wires[1])
        q.Hadamard(wires=wires[1])
    
    # phase estimation
    qa.Phase_Estimation(U,t=t,wires=wires)
    
    return q.probs(wires)

# QNode
circuit = q.QNode(func,dev)

In [27]:
U = np.array([[0,1],
              [1,0]],dtype='complex128')

In [28]:
# Check
states = c.states_vector(wires)
measurements = circuit(U,2)
np.vstack([states,measurements]).T

array([['|00>', '0.0'],
       ['|01>', '0.0'],
       ['|10>', '0.499548'],
       ['|11>', '0.500452']], dtype='<U32')

In [29]:
start = datetime.datetime.now()

# check
df = pd.DataFrame(c.states_vector(wires),columns=['states'])
for i in range(1,len(wires)+1):
    
    measurements = circuit(U,i)
    
    df['u_'+str(i)] = measurements

end = datetime.datetime.now()
print(end-start)

0:00:00.180152


In [30]:
df

,states,u_1,u_2
0,|00>,0.50011,0.000000
1,|01>,0.49989,0.000000
2,|10>,0.00000,0.499669
3,|11>,0.00000,0.500331


#### Setting 2
First register contains 2 qubits\
Second register contains 2 qubits
$$U = \begin{bmatrix} 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \\ \end{bmatrix}$$
The first register is prepared in a state$|00\rangle$\
The second register is prepared in one of four eigenstates of U:
$$|u_1\rangle = \frac{1}{2}\begin{bmatrix} 1 \\ 1 \\ 1 \\ 1 \end{bmatrix}\text{ , }|u_2\rangle = \frac{1}{2}\begin{bmatrix} 1 \\ 1 \\ -1 \\ -1 \end{bmatrix}\text{ , }|u_3\rangle = \frac{1}{2}\begin{bmatrix} 1 \\ -1 \\ 1 \\ -1 \end{bmatrix}\text{ , }|u_4\rangle = \frac{1}{2}\begin{bmatrix} 1 \\ -1 \\ -1 \\ 1 \end{bmatrix}\text{ , }$$
Actual measurements are performed for $|u_1\rangle$ and $|u_3\rangle$

In [31]:
t = 2

In [32]:
# wires
wires=['q0','q1','q2','q3']
# device
dev = q.device('default.qubit', wires=wires, shots=1e6, analytic=False)

# circuit
def func(U,eigenstate):
    
    # preparation
    if eigenstate.val == 1:
        q.Hadamard(wires=wires[2])
        q.Hadamard(wires=wires[3])
    if eigenstate.val == 3:
        q.PauliX(wires=wires[2])
        q.Hadamard(wires=wires[2])
        q.PauliX(wires=wires[3])
        q.Hadamard(wires=wires[3])
    
    # phase estimation
    qa.Phase_Estimation(U,t=t,wires=wires)
    
    return q.probs(wires)

# QNode
circuit = q.QNode(func,dev)

In [33]:
U = np.array([[0,1,0,0],
              [1,0,0,0],
              [0,0,0,1],
              [0,0,1,0]],dtype='complex128')

In [34]:
# Check input
states = c.states_vector(wires)
measurements = circuit(U,1)
np.vstack([states,measurements]).T

array([['|0000>', '0.249891'],
       ['|0001>', '0.249612'],
       ['|0010>', '0.250367'],
       ['|0011>', '0.25013'],
       ['|0100>', '0.0'],
       ['|0101>', '0.0'],
       ['|0110>', '0.0'],
       ['|0111>', '0.0'],
       ['|1000>', '0.0'],
       ['|1001>', '0.0'],
       ['|1010>', '0.0'],
       ['|1011>', '0.0'],
       ['|1100>', '0.0'],
       ['|1101>', '0.0'],
       ['|1110>', '0.0'],
       ['|1111>', '0.0']], dtype='<U32')

In [35]:
start = datetime.datetime.now()

# check
df = pd.DataFrame(c.states_vector(wires),columns=['states'])

measurements = circuit(U,1)
df['u_1'] = measurements
measurements = circuit(U,3)
df['u_3'] = measurements

end = datetime.datetime.now()
print(end-start)

0:00:00.631671


In [36]:
df

,states,u_1,u_3
0,|0000>,0.249460,0.000000
1,|0001>,0.250724,0.000000
2,|0010>,0.250007,0.000000
3,|0011>,0.249809,0.000000
4,|0100>,0.000000,0.000000
5,|0101>,0.000000,0.000000
6,|0110>,0.000000,0.000000
7,|0111>,0.000000,0.000000
8,|1000>,0.000000,0.250445
9,|1001>,0.000000,0.249743


## Commit comments